## Kaggle Titanic
１．欠損値の補完  
２．文字型データの数値型データへの変換  
３．不要な列の削除  

In [152]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import np_utils

%matplotlib inline

import check_miss_value

In [153]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [154]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [155]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [156]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [157]:
train.Sex.values.reshape(-1, 1).shape

(891, 1)

In [158]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [159]:
from sklearn.preprocessing import LabelEncoder

In [160]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [161]:
LE1 = LabelEncoder()

In [162]:
train.Sex = LE1.fit_transform(train.Sex.values.reshape(-1, 1))

In [163]:
train['Sex'].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [164]:
train.Sex.value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [165]:
# train.Ticket.value_counts()

In [166]:
# Ticket は後で削除する

In [167]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [168]:
LE2 = LabelEncoder()

In [169]:
from sklearn.preprocessing import Imputer

In [170]:
impubter = Imputer(missing_values=np.nan, strategy='most_frequent')

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [171]:
# 文字列型は Imputerで入れられないので、最頻値を直接入れる
train.Embarked = train.Embarked.fillna('S')

In [172]:
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [173]:
train.Embarked = LE2.fit_transform(train.Embarked.values.reshape(-1, 1))

In [174]:
train.Embarked.value_counts()

2    646
0    168
1     77
Name: Embarked, dtype: int64

In [175]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.057527,-0.001652,0.012658,0.013128
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,-0.035322,0.081629,0.257307,-0.167675
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,0.083081,0.018443,-0.549500,0.162098
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114631,-0.245489,-0.182333,0.108262
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,-0.308247,-0.189119,0.096067,-0.030394
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.308247,1.000000,0.414838,0.159651,0.068230
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.189119,0.414838,1.000000,0.216225,0.039798
Fare,0.012658,0.257307,-0.549500,-0.182333,0.096067,0.159651,0.216225,1.000000,-0.224719
Embarked,0.013128,-0.167675,0.162098,0.108262,-0.030394,0.068230,0.039798,-0.224719,1.000000


In [176]:
train.corr()[train.corr() > 0.2]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.257307,NaN
Pclass,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Sex,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Age,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,NaN,1.000000,0.414838,NaN,NaN
Parch,NaN,NaN,NaN,NaN,NaN,0.414838,1.000000,0.216225,NaN
Fare,NaN,0.257307,NaN,NaN,NaN,NaN,0.216225,1.000000,NaN
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [177]:
train.corr()[train.corr() < -0.2]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,NaN,-0.338481,-0.543351,NaN,NaN,NaN,NaN,NaN
Pclass,NaN,-0.338481,NaN,NaN,-0.369226,NaN,NaN,-0.549500,NaN
Sex,NaN,-0.543351,NaN,NaN,NaN,NaN,-0.245489,NaN,NaN
Age,NaN,NaN,-0.369226,NaN,NaN,-0.308247,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,-0.308247,NaN,NaN,NaN,NaN
Parch,NaN,NaN,NaN,-0.245489,NaN,NaN,NaN,NaN,NaN
Fare,NaN,NaN,-0.549500,NaN,NaN,NaN,NaN,NaN,-0.224719
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.224719,NaN


In [178]:
# PassengerId, Name, Ticket, Cabin　を削除
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [179]:
check_miss_value.check_miss_value(train)

,欠損数,%
Survived,0,0.00000
Pclass,0,0.00000
Sex,0,0.00000
Age,177,19.86532
SibSp,0,0.00000
Parch,0,0.00000
Fare,0,0.00000
Embarked,0,0.00000


In [180]:
train.Age = train.Age.fillna(train.Age.median())

In [181]:
check_miss_value.check_miss_value(train)

,欠損数,%
Survived,0,0.0
Pclass,0,0.0
Sex,0,0.0
Age,0,0.0
SibSp,0,0.0
Parch,0,0.0
Fare,0,0.0
Embarked,0,0.0


In [182]:
# testデータ
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


In [183]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [184]:
test.Age.median()

27.0

In [185]:
test.Age = test.Age.fillna(test.Age.median())

In [186]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [187]:
test.Fare = test.Fare.fillna(test.Fare.median())

In [188]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [189]:
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000
Cabin,327,78.229665


In [190]:
# Name, Ticket, Cabin　を削除
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [192]:
test.Sex = LE1.fit_transform(test.Sex.values.reshape(-1, 1))

In [193]:
test.Embarked = LE2.fit_transform(test.Sex.values.reshape(-1, 1))

In [195]:
test.Sex.value_counts()

1    266
0    152
Name: Sex, dtype: int64

In [196]:
test.Embarked.value_counts()

1    266
0    152
Name: Embarked, dtype: int64

In [197]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int32
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null int64
dtypes: float64(2), int32(1), int64(5)
memory usage: 24.6 KB


In [198]:
test.corr()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.000000,-0.026751,0.023245,-0.031447,0.003818,0.043080,0.008633,0.023245
Pclass,-0.026751,1.000000,0.108615,-0.467853,0.001087,0.018721,-0.577313,0.108615
Sex,0.023245,0.108615,1.000000,-0.008035,-0.099943,-0.159120,-0.192036,1.000000
Age,-0.031447,-0.467853,-0.008035,1.000000,-0.071197,-0.043731,0.342357,-0.008035
SibSp,0.003818,0.001087,-0.099943,-0.071197,1.000000,0.306895,0.171912,-0.099943
Parch,0.043080,0.018721,-0.159120,-0.043731,0.306895,1.000000,0.230325,-0.159120
Fare,0.008633,-0.577313,-0.192036,0.342357,0.171912,0.230325,1.000000,-0.192036
Embarked,0.023245,0.108615,1.000000,-0.008035,-0.099943,-0.159120,-0.192036,1.000000


In [199]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null int32
dtypes: float64(2), int32(2), int64(4)
memory usage: 48.8 KB


In [200]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int32
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null int64
dtypes: float64(2), int32(1), int64(5)
memory usage: 24.6 KB


In [201]:
# picle ファイルに出力
train.to_pickle('pd_train.pk2')
test.to_pickle('pd_test.pk2')

In [204]:
train.Age.min(), train.Age.max(), test.Age.min(), test.Age.max()

(0.42, 80.0, 0.17, 76.0)

In [202]:
# 年齢を 10 最刻みのレンジにする
train['Age_bin'] = train['Age'] // 10
test['Age_bin'] = test['Age'] // 10

In [205]:
train.Fare.min(), train.Fare.max(), test.Fare.min(), test.Fare.max()

(0.0, 512.3292, 0.0, 512.3292)

In [206]:
# 運賃を 20 $ 毎のレンジにする
train['Fare_bin'] = train['Fare'] // 25
test['Fare_bin'] = test['Fare'] // 25